https://m.ensembl.org/info/docs/api/core/core_schema.html#associated_group

gene: all
xref: display_label (gene name) and description

In [1]:
%load_ext blackcellmagic

In [2]:
db = "homo_sapiens_core_105_38"
searchword = "GABA"
limit = 20

In [3]:
import pandas as pd
# pure-Python MySQL client library
import pymysql

database = pymysql.connect(
host="ensembldb.ensembl.org",
user="anonymous",
password="",
database=db
)

In [244]:
cursor = database.cursor()

# ## Select genes based on search word in "description" column of "gene" table
# cursor.execute(f"select * from gene where description like '%{searchword}%';")
# res1 = cursor.fetchall()

# # Get gene table column names
# cursor.execute(f"show columns from gene;")
# header_tuples = cursor.fetchall()
# header1 = [a_tuple[0] for a_tuple in header_tuples]

# # Save search results from gene table in data frame
# df_temp1 = pd.DataFrame(
#     res1,
#     columns=header1,
# )

# # "display_xref_id" links to "xref_id" in "xref" table
# display_xref_ids = df_temp["display_xref_id"].values

# # Find search results form gene table in xref table
# for xref_id in display_xref_ids:
#     cursor.execute(f"select external_db_id, description from xref where xref_id like '%{xref_id}%';")
#     res2 = cursor.fetchall()
#     header2 = ["ext database ID", "ext description"]


cursor.execute(f"select description from gene where description like '%{searchword}%';")
gene_description = cursor.fetchall()

## Select genes based on search word in "description" column of "xref" table
# This table contains external web-data displayed on the Ensembl website
cursor.execute(f"select description from xref where description like '%{searchword}%';")
ext_description = cursor.fetchall()
# header2 = ["ext database ID", "ext description"]


# # external_db_ids = 
# ext_dbs = []
# for ext_id in external_db_ids:
#     ## Get database name of xref genes from "db_name" column of "external_db" table
#     cursor.execute(f"select db_name from external_db where external_db_id like '%{external_db_ids}%';")
#     ext_dbs.append

## Get gene names from "value" column in gene_attrib table
cursor.execute(f"select value from gene_attrib limit {limit};")
gene_names = cursor.fetchall()

header3 = ["gene_name"]

In [249]:
len(gene_description)

16

In [248]:
len(ext_description)

350

In [7]:
cursor = database.cursor()
# Union between left and right join replaces full outer join in this vesion of SQL
cursor.execute(
    f"""
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene 
    LEFT JOIN xref ON gene.display_xref_id=xref.xref_id 
    WHERE gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%'
    UNION
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene   
    RIGHT JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
    WHERE gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%';
    """
)

results = cursor.fetchall()

# Get gene table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]
# Define column name for xref description and gene_attrib.value
header2 = ["xref description", "gene name"]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

In [32]:
cursor = database.cursor()
# Union between left and right join replaces full outer join in this vesion of SQL
cursor.execute(
    f"""
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene 
    LEFT JOIN xref ON gene.display_xref_id=xref.xref_id 
    LEFT JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
    WHERE gene_attrib.attrib_type_id LIKE '4' AND gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%'
    UNION
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene 
    RIGHT JOIN xref ON gene.display_xref_id=xref.xref_id 
    RIGHT JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
    WHERE gene_attrib.attrib_type_id LIKE '4' AND gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%';
    """
)

results = cursor.fetchall()

# Get gene table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]
# Define column name for xref description and gene_attrib.value
header2 = ["xref description", "gene name"]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

In [33]:
df_temp

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,is_current,canonical_transcript_id,stable_id,version,created_date,modified_date,xref description,gene name
0,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,RP11-656E20.5
1,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,38.10
2,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,OTTHUMG00000168529
3,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,processed_transcript
4,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,GABARAPL1
5,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,44.11
6,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,OTTHUMG00000168411
7,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,ENST00000266458
8,20765,processed_pseudogene,8354,131546,32072734,32072880,1,NaN,havana,"gamma-aminobutyric acid (GABA) A receptor, pi ...",1,63959,ENSG00000257852,1,2011-05-11 16:49:31,2011-05-11 16:49:31,None,RP11-843B15.1
9,21715,processed_pseudogene,8354,131560,14690382,14690619,1,NaN,havana,diazepam binding inhibitor (GABA receptor modu...,1,66907,ENSG00000255365,1,2010-11-01 15:31:55,2010-11-01 15:31:55,None,RP11-776K3.1


In [30]:
table = "gene_attrib"

cursor = database.cursor()
cursor.execute(
    f"""
    SELECT *
    FROM {table}
    WHERE gene_attrib.attrib_type_id LIKE '4'
    LIMIT 10;
    """
)

results = cursor.fetchall()

# Get table column names
cursor.execute(f"SHOW COLUMNS FROM {table};")
header_tuples = cursor.fetchall()
header = [a_tuple[0] for a_tuple in header_tuples]

# Save search results in data frame
df_temp = pd.DataFrame(
    results,
    columns = header,
)

df_temp

,gene_id,attrib_type_id,value
0,23150,4,RP11-165F24.3
1,12474,4,CCDC85C
2,12535,4,GABRB1
3,12557,4,RP11-380P13.1
4,11925,4,LINGO2
5,33855,4,CTD-2325P2.2
6,21883,4,DEFB104A
7,23775,4,RP11-531A21.2
8,12945,4,NCRNA00163
9,12950,4,LINC00165


In [264]:
cursor = database.cursor()
cursor.execute(
    f"""
    SELECT gene.*, gene_attrib.* 
    FROM gene 
    INNER JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
    WHERE gene.description LIKE '%{searchword}%';
    """
)
results = cursor.fetchall()

# Get table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]

cursor.execute(f"show columns from gene_attrib;")
header_tuples = cursor.fetchall()
header2 = [a_tuple[0] for a_tuple in header_tuples]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

In [268]:
len(df_temp)

56

Why is len(df_temp) != len(gene_description) here???  
It looks like gene_attrib has multiple entries for the same gene ID with different entries for "value". Let's try excluding gene_attrib.  
Note: Are "na" values in display_xref_id a problem?

In [269]:
cursor = database.cursor()
cursor.execute(
    f"""
    SELECT gene.*, xref.description
    FROM gene 
    INNER JOIN xref ON gene.display_xref_id=xref.xref_id 
    WHERE gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%';
    """
)
results = cursor.fetchall()

# Get table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]

header2 = ["xref description"]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

In [270]:
len(df_temp)

7

In [271]:
df_temp

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,is_current,canonical_transcript_id,stable_id,version,created_date,modified_date,xref description
0,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1
1,18340,protein_coding,8353,131546,10212458,10223128,1,6355576,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1
2,44845,protein_coding,8353,131549,75566375,75577881,1,6355579,ensembl_havana,GABA type A receptor associated protein like 2...,1,149153,ENSG00000034713,8,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 2
3,53729,TEC,8354,131558,90347587,90349437,1,4017258,havana,GABA type A receptor associated protein like 3...,1,187564,ENSG00000279980,1,2014-08-13 17:29:05,2014-08-13 17:29:05,GABA type A receptor associated protein like 3...
4,53730,processed_pseudogene,8354,131558,90348844,90349197,-1,6355582,havana,GABA type A receptor associated protein like 3...,1,187568,ENSG00000238244,3,2009-05-19 09:47:17,2014-06-09 10:49:07,GABA type A receptor associated protein like 3...
5,60597,protein_coding,8353,131537,44954585,44993341,1,6345391,ensembl_havana,CLPTM1 regulator of GABA type A receptor forwa...,1,223716,ENSG00000104853,16,2008-04-29 11:17:41,2018-11-21 17:23:49,CLPTM1 regulator of GABA type A receptor forwa...
6,62468,protein_coding,8353,131554,7240008,7242449,-1,6355574,ensembl_havana,GABA type A receptor-associated protein [Sourc...,1,233989,ENSG00000170296,10,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor-associated protein
